# Assignment 7
Elements of Data Science and Artificial Intelligence
<br>
Winter Semester 2022/2023
<br>
Saarland University

Prof. Dr. Jens Dittrich
<br>
Prof. Dr. Jörg Hoffmann
<br>
Prof. Dr. Bernt Schiele
<br>
Dr. Sebastian Schuster

#### Submission Details
Upload your submission to our [CMS](https://cms.sic.saarland/edsai2223/) in groups accroding to the team grouping until **January 22, 2023 23:59**. Late submissions will not be graded! The submission should be uploaded by exactly **one** team member. Make sure that your submission contains the name and matriculation number of each team member. For this purpose, fill out the cell below.

In [1]:
# Team information (name - matriculation number)
# Felix Ceard-Falkenberg - 700490
# Please remove other teammates from submition - no contact since Sheet 2

## Connect Four

In this exercise, we will consider the well-known two-player game *Connect Four*. In this game, two players Max and Min alternatingly drop a token into a suspended grid. Here, we assume a 6x6 grid. Tokens can be dropped into any column that is not completely filled with tokens already, in which case the tokens falls down to the lowest unoccupied cell of the column. We assume that Max has **X** tokens and Min has **O** tokens. The goal of the game -- as the name implies -- is to build a string of four connected tokens horizontally, vertically, or diagonally. The player to do so first wins the game. If no player can achieve a string of four tokens and the grid is completely filled, then the game is drawn.

We will represent a game state of Connect Four as an instance of the following class `State`.

- The member `next_token` represents the next token to place / the next player to move (either `'X'` or `'O'`).
- The member `grid` represents the current state of the grid as simple 6x6 matrix (a list of lists) of tokens `'X'`, `'O'` or `'_'` (blank). We identify each grid cell with a 2D coordinate, where the origin $\langle0, 0\rangle$ is rooted at the top left of the grid (the y-axis points **downwards**). Instead of accessing `grid` directly, the cell at coordinate $\langle x, y \rangle$ can also be accessed by `state[x, y]`.

On the other hand, a move is simply represented by the integer index of the column in which the token (**X** or **O** as given by `state.next_token`) is to be placed, i.e., a number between 0 and 5 (inclusive).

In [2]:
import copy

# Constants for the board dimensionality.
# (Using these is not strictly necessary, they may be hardcoded)
GRID_WIDTH = 6
GRID_HEIGHT = 6


# Do not change!
class State:
    def __init__(self, next_token, grid):
        assert (next_token in ['X', 'O'])
        assert (len(grid) == GRID_WIDTH and (len(grid[i]) == 3 for i in range(0, GRID_HEIGHT)))

        self.next_token = next_token
        self.grid = grid

        for x in range(0, GRID_WIDTH):
            for y in range(0, GRID_HEIGHT):
                assert (self[x, y] in ['_', 'X', 'O'])

    # Allows reading access via *state[x, y]*
    def __getitem__(self, ij):
        i, j = ij
        return self.grid[i][j]

    # Allows writing access via *state[x, y] = z*
    def __setitem__(self, ij, z):
        i, j = ij
        self.grid[i][j] = z

    # Can be ignored
    def copy(self):
        return State(self.next_token, copy.deepcopy(self.grid))

    # Can be ignored
    def __eq__(self, other):
        return self.next_token == other.next_token and (self.grid[i] == other.grid[i] for i in range(0, GRID_HEIGHT))

## Exercise 4: Minimax Implementation (5 Points)

Your task in this exercise is to implement the Minimax algorithm to play a game of Connect Four.

In [3]:
# define the unittests
% run -i assignment07_unittests

### a) Utility Function (2.5 Points)

First, you will define the utility function for Connect Four. To this end, implement the function `terminal_test(s)`, which checks if the state `s` is terminal, i.e. the game is over, and if yes returns the utility. If not, this function should return `None`. The utility of a terminal state is +100 if the Max player (**X**) won and -100 if the Min player (**O**) won, whereas a draw yields utility 0.

In [178]:
def terminal_test(s: State):
    def win(node):
        return 100 if node == 'X' else -100

    grid_width = len(s.grid)
    grid_height = len(s.grid[0])

    # Horizontal
    for i in range(grid_width):
        node_type = None
        streak_length = 0
        for j in range(grid_height):
            if s[j, i] == '_':
                node_type = None
                streak_length = 0
                continue

            if s[j, i] == node_type:
                streak_length += 1
            else:
                node_type = s[j, i]
                streak_length = 1

            if streak_length == 4:
                return win(node_type)

    # Vertical
    for i in range(grid_height):
        node_type = None
        streak_length = 0
        for j in range(grid_width):
            if s[i, j] == '_':
                node_type = None
                streak_length = 0
                continue

            if s[i, j] == node_type:
                streak_length += 1
            else:
                node_type = s[i, j]
                streak_length = 1

            if streak_length == 4:
                return win(node_type)

    # Diagonal
    # Top left to bottom right
    for i in range(grid_width - 4 + 1):
        for j in range(grid_height - 4 + 1):
            node_type = s[j, i]
            if node_type == '_':
                continue
            found_four = True
            for d in range(4):
                if node_type != s[j + d, i + d]:
                    found_four = False
                    break
            if found_four:
                return win(node_type)

    # Bottom left to top right
    for i in range(grid_width - 4 + 1):
        for j in range(grid_height - 4 + 1, grid_height):
            node_type = s[j, i]
            if node_type == '_':
                continue
            found_four = True
            for d in range(4):
                if node_type != s[j - d, i + d]:
                    found_four = False
                    break
            if found_four:
                return win(node_type)

    # test full:
    for i in range(len(s.grid)):
        for j in range(len(s.grid[0])):
            if s[i, j] == '_':
                return None

    return 0

In [179]:
# Run tests
unittest.main(argv=['ignored', '-v', 'Interface_Test'], verbosity=2, exit=False)

test_blank_not_terminal (__main__.Interface_Test) ... ok
test_complete_min_wins (__main__.Interface_Test) ... ok
test_draw (__main__.Interface_Test) ... ok
test_incomplete_max_wins_diagonal (__main__.Interface_Test) ... ok
test_incomplete_max_wins_horizontal (__main__.Interface_Test) ... ok
test_incomplete_max_wins_vertical (__main__.Interface_Test) ... ok
test_incomplete_min_wins_diagonal (__main__.Interface_Test) ... ok
test_incomplete_min_wins_horizontal (__main__.Interface_Test) ... ok
test_incomplete_min_wins_vertical (__main__.Interface_Test) ... ok
test_one_mark_not_terminal (__main__.Interface_Test) ... ok
test_three_marks_not_terminal (__main__.Interface_Test) ... ok

----------------------------------------------------------------------
Ran 11 tests in 0.009s

OK


### b) The Minimax Algorithm (2.5 Points)

Next, you will implement the actual Minimax algorithm. To do so, first take a look at the pseudocode depicted on the lecture slides for Minimax (slide 15).

We slightly deviate from the pseudocode in this exercise: Implement functions `Max_Value_Decision` and `Min_Value_Decision` that return a *pair* $\langle v, a \rangle$ of the optimal utility value $v$ *plus the 
action $a$ that yields this utility* for the Max and Min player respectively (return `None` for `a` if the state is terminal). Afterwards, implement `Minimax`, which should return the move for Max that yields the highest utility.

We already provide you with the function `child_state(s, a)` and `action(s)` below which respectively construct the successor state of `s` when appling the move `x` and list the applicable action when in state `s`. Use these functions in your implementation.

*Hints: Note that your implementation of `terminal_test` also returns the utility of a terminal state. The constant `math.inf` might be useful to compute the maximum / minimum.*

In [180]:
import math


# Successor Generation. Constructs the successor state of a state when applying
# a specific (legal) move. Use, but do not change!
def child_state(s, x):
    ns = s.copy()

    mark = ns.next_token

    for y in range(GRID_HEIGHT - 1, -1, -1):
        if ns[x, y] == '_':
            ns[x, y] = mark
            break

    ns.next_token = 'X' if mark == 'O' else 'O'

    return ns


# Applicable action generation. Returns the list of moves (legal columns) for a
# state. Use, but do not change!
def actions(s):
    acts = []

    for x in range(0, GRID_WIDTH):
        for y in range(0, GRID_HEIGHT):
            if s[x, y] == '_':
                acts.append(x)
                break

    return acts


def Max_Value_Decision(s, initial=False):
    if terminal_test(s) != None:
        return (terminal_test(s), None)

    v = float('-inf')
    max_action = None
    for action in actions(s):
        v_min, _ = Min_Value_Decision(child_state(s, action))
        if initial:
            print(action, v_min, v)
        if v_min > v:
            max_action = action
        v = max(v, v_min)

    return (v, max_action)


def Min_Value_Decision(s, initial=False):
    if terminal_test(s) != None:
        return (terminal_test(s), None)

    v = float('inf')
    max_action = None
    for action in actions(s):
        v_min, _ = Max_Value_Decision(child_state(s, action))
        if initial:
            print(action, v_min, v)
        if v_min < v:
            max_action = action
        v = min(v, v_min)

    return (v, max_action)


def Minimax_Decision(s):
    v = float('-inf')
    max_action = None
    for action in actions(s):
        v_min, _ = Max_Value_Decision(child_state(s, action))
        if v_min > v:
            max_action = action
        v = max(v, v_min)

    return (v, max_action)

state = State('O', [
            ['_', 'O', 'O', 'X', 'X', 'X'],
            ['O', 'X', 'X', 'O', 'O', 'O'],
            ['O', 'O', 'O', 'X', 'X', 'X'],
            ['_', 'X', 'X', 'O', 'X', 'O'],
            ['O', 'X', 'O', 'O', 'X', 'O'],
            ['_', 'O', 'X', 'X', 'O', 'X']
        ])

v, a = Min_Value_Decision(state, True)
v, a

0 100 inf
3 -100 100
5 0 -100


(-100, 3)

In [181]:
# Run tests
unittest.main(argv=['ignored', '-v', 'Minimax_Test'], verbosity=2, exit=False)

test_minimax_draw_if_perfect_play (__main__.Minimax_Test) ... ok
test_minimax_max_can_win (__main__.Minimax_Test) ... ok
test_minimax_min_can_win (__main__.Minimax_Test) ... ok
test_minimax_obvious_max_always_loses (__main__.Minimax_Test) ... ok
test_minimax_obvious_min_can_win (__main__.Minimax_Test) ... ok

----------------------------------------------------------------------
Ran 5 tests in 8.784s

OK


## Final Remarks
The Minimax algorithm is an extremely simple and inefficient game playing algorithm, so your implementation will only be able to solve states in which the game has already progressed quite far. Besides those techniques covered in the lecture, there is also [Alpha-Beta Pruning](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning), a more sophisticated extension of Minimax that can avoid the exploration of some parts of the search space.